# User-based recommendation on user clusters (based on likes)

## Import

In [2]:
import pandas as pd
import re
import ast
#from transformers import pipeline

In [134]:
data_reviews = pd.read_csv("https://anime-recommendation-engine.s3.eu-west-3.amazonaws.com/data/reviews_clean.csv")

with pd.option_context('display.max_colwidth', None):
    display(data_reviews.head(1))

uid         profile  anime_uid  \
0  255938  DesolatePsyche      34096   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

## Preprocessing

In [107]:
def clean_and_split_text(text):
    text = re.sub("[\n\r]", "", text)
    text = re.sub("[^A-Za-z0-9]+", " ", text)

    text_splited = re.findall('.{512}', text)
    text_splited.append(text[512 * len(text_splited):])

    return text_splited

In [108]:
def compute_score_from_classifications(classifications):
    
    score = 0

    for classification in classifications :

        if classification["label"] == "POSITIVE" :
            score += classification["score"]
        else :
            score -= classification["score"]

    normalize_score = (score / len(classifications) + 1) / 2

    return normalize_score

In [109]:
sentiment_pipeline = pipeline(task="sentiment-analysis", model="distilbert/distilbert-base-uncased-finetuned-sst-2-english")

Device set to use mps:0


In [110]:
df_als_reviews_score = data_reviews[["profile", "anime_uid", "text", "score"]].copy()

In [111]:
df_als_reviews_score["text"] = df_als_reviews_score["text"].transform(clean_and_split_text)

In [140]:
# df_als_reviews_score["classifications"] = df_als_reviews_score["text"].transform(sentiment_pipeline)
df_als_reviews_score = pd.read_csv("../data/als_reviews_score_raw.csv")

In [141]:
df_reviews_10 = df_als_reviews_score["anime_uid"].value_counts().reset_index(name="count")
df_reviews_10 = df_reviews_10.loc[df_reviews_10["count"]>=10,"anime_uid"]

df_als_reviews_score = df_als_reviews_score.merge(df_reviews_10, on="anime_uid", how="inner")
df_als_reviews_score["classifications"] = df_als_reviews_score["classifications"].apply(ast.literal_eval)
df_als_reviews_score["review_score"] = df_als_reviews_score["classifications"].transform(compute_score_from_classifications)

In [129]:
df_als_reviews_score["delta"] = abs(df_als_reviews_score["score"]-df_als_reviews_score["review_score"]*10)

In [130]:
df_als_reviews_score

,profile,anime_uid,text,score,classifications,review_score,delta
0,DesolatePsyche,34096,[' more pics Overall 8 Story 8 Animation 8 Sou...,8,"[{'label': 'POSITIVE', 'score': 0.931263029575...",0.709280,0.907201
1,baekbeans,34599,[' more pics Overall 10 Story 10 Animation 10 ...,10,"[{'label': 'POSITIVE', 'score': 0.882984995841...",0.786679,2.133211
2,skrn,28891,[' more pics Overall 7 Story 7 Animation 9 Sou...,7,"[{'label': 'POSITIVE', 'score': 0.972522079944...",0.653450,0.465503
3,edgewalker00,2904,[' more pics Overall 9 Story 9 Animation 9 Sou...,9,"[{'label': 'NEGATIVE', 'score': 0.999156832695...",0.559802,3.401976
4,aManOfCulture99,4181,[' more pics Overall 10 Story 10 Animation 8 S...,10,"[{'label': 'POSITIVE', 'score': 0.998817861080...",0.750125,2.498746
...,...,...,...,...,...,...,...
130514,iHitokage,2593,[' more pics Overall 10 Story 10 Animation 10 ...,10,"[{'label': 'POSITIVE', 'score': 0.913590371608...",0.985183,0.148172
130515,Onegai,2593,[' more pics Overall 7 Story 6 Animation 9 Sou...,7,"[{'label': 'NEGATIVE', 'score': 0.882237493991...",0.471138,2.288619
130516,Murasa22,2593,[' more pics Overall 10 Story 9 Animation 10 S...,10,"[{'label': 'POSITIVE', 'score': 0.968979001045...",0.539184,4.608158
130517,srry4apologizng,2593,[' more pics Overall 9 Story 5 Animation 10 So...,9,"[{'label': 'POSITIVE', 'score': 0.999805629253...",0.611100,2.889000


In [142]:
df_als_reviews_score[["profile", "anime_uid", "review_score"]].to_csv("../data/als_reviews_score_10.csv", index=False)

## Spark

For now, this part have to be executed in a ad-hoc Jupyter environment with PySpark, following those steps (disclaimer : you need to install and configure PySpark first) :


```shell
pyspark --name anime-recommendation-engine --driver-java-options -Djava.security.manager=allow
```

```python
sc = SparkSession.builder.getOrCreate()
```

[Medium article simple ALS](https://medium.com/@patelneha1495/recommendation-system-in-python-using-als-algorithm-and-apache-spark-27aca08eaab3)

[Medium article advanced ALS](https://medium.com/@brunoborges_38708/recommender-system-using-als-in-pyspark-10329e1d1ee1)


In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, rank
from pyspark.sql.window import Window

from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import StringIndexer
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

In [6]:
sc = SparkSession.builder.getOrCreate()

In [24]:
df_als_reviews_score = pd.read_csv("../data/als_reviews_score_50.csv")

display(df_als_reviews_score.head(2)) # (130519, 3)

,profile,anime_uid,review_score
0,baekbeans,34599,0.786679
1,skrn,28891,0.653450


In [26]:
def pyspark_df_with_train_test_split(df, test_size, user_col, item_col, rating_col) :

    df_items_counted = df.groupby(user_col)[rating_col].count().reset_index(name="num_items")
    df_merged = df.merge(df_items_counted, on=user_col)

    user_window = Window.partitionBy(user_col).orderBy(col(item_col).desc())

    df_spark = sc.createDataFrame(df_merged)
    df_spark = df_spark.withColumn("num_items_to_mask", (col("num_items") * test_size).cast("int"))
    df_spark = df_spark.withColumn("item_rank", rank().over(user_window))

    indexer = StringIndexer(inputCol=user_col, outputCol=user_col+"_index")

    df_spark = indexer.fit(df_spark).transform(df_spark)

    training = df_spark.filter(col("item_rank") > col("num_items_to_mask"))
    test = df_spark.filter(col("item_rank") <= col("num_items_to_mask"))

    return df_spark, training, test 

In [30]:
def als_tuning_and_predict(train, test, user_col, item_col, rating_col) :

    als = ALS(userCol=user_col+"_index", itemCol=item_col, ratingCol=rating_col,
            coldStartStrategy="drop", nonnegative=True)

    param_grid = ParamGridBuilder()\
                .addGrid(als.rank, [10, 15])\
                .addGrid(als.maxIter, [10])\
                .addGrid(als.regParam, [.05, .1, .15])\
                .build()
    
    evaluator = RegressionEvaluator(metricName="rmse", labelCol=rating_col, predictionCol="prediction")

    cv = CrossValidator(
            estimator=als,
            estimatorParamMaps=param_grid,
            evaluator=evaluator,
            numFolds=3)
    
    model = cv.fit(train)

    print("Rank: ", model.bestModel.rank)
    print("MaxIter: ", model.bestModel._java_obj.parent().getMaxIter())
    print("RegParam: ", model.bestModel._java_obj.parent().getRegParam())

    predictions = model.bestModel.transform(test)
    rmse = evaluator.evaluate(predictions)
    
    print("RMSE="+str(rmse))

    return model.bestModel

In [28]:
def als_tuning_and_predict(train, test, user_col, item_col, rating_col) :

    als = ALS(rank=10, maxIter=10,regParam=.1, userCol=user_col+"_index", itemCol=item_col, ratingCol=rating_col,
            coldStartStrategy="drop", nonnegative=True)

    evaluator = RegressionEvaluator(metricName="rmse", labelCol=rating_col, predictionCol="prediction")

    model = als.fit(train)
    
    predictions = model.transform(test)
    rmse = evaluator.evaluate(predictions)
    
    print("RMSE="+str(rmse))

    return model

In [30]:
def create_predictions_pandas_dataframe(model, df_spark, user_col, item_col, rating_col) :
    
    recs = model.recommendForAllUsers(10).toPandas()

    nrecs = recs.recommendations.apply(pd.Series)\
                .merge(recs, right_index = True, left_index = True)\
                .drop(["recommendations"], axis = 1)\
                .melt(id_vars = [user_col+"_index"], value_name = "recommendation")\
                .drop("variable", axis = 1)\
                .dropna()
    nrecs = nrecs.sort_values(user_col+"_index")
    nrecs = pd.concat([nrecs["recommendation"].apply(pd.Series), nrecs[user_col+"_index"]], axis = 1)
    nrecs.columns = [item_col, rating_col, user_col]

    md = df_spark.select(df_spark[user_col],df_spark[user_col+"_index"],df_spark[item_col])
    md = md.toPandas()

    dict1 = dict(zip(md[user_col+"_index"],md[user_col]))

    nrecs[user_col] = nrecs[user_col].map(dict1)
    nrecs = nrecs.sort_values(user_col)
    nrecs.reset_index(drop=True, inplace=True)

    df_recommendations = nrecs[[user_col,item_col]]
    df_recommendations[item_col] = df_recommendations[item_col].astype("int64")

    df_recommendations = df_recommendations[item_col].groupby([df_recommendations[user_col]]).apply(list).reset_index()

    return df_recommendations

### Reviews score

In [32]:
df_sp_reviews_score, train, test = pyspark_df_with_train_test_split(df_als_reviews_score, 0.2, "profile", "anime_uid", "review_score")

In [34]:
model = als_tuning_and_predict(train, test, "profile", "anime_uid", "review_score")

25/06/05 10:01:20 WARN DAGScheduler: Broadcasting large task binary with size 1224.3 KiB
25/06/05 10:01:20 WARN DAGScheduler: Broadcasting large task binary with size 1225.7 KiB
25/06/05 10:01:20 WARN DAGScheduler: Broadcasting large task binary with size 1227.3 KiB
25/06/05 10:01:20 WARN DAGScheduler: Broadcasting large task binary with size 1228.6 KiB
25/06/05 10:01:21 WARN DAGScheduler: Broadcasting large task binary with size 1227.5 KiB
25/06/05 10:01:21 WARN DAGScheduler: Broadcasting large task binary with size 1228.8 KiB
25/06/05 10:01:21 WARN DAGScheduler: Broadcasting large task binary with size 1229.6 KiB
25/06/05 10:01:21 WARN DAGScheduler: Broadcasting large task binary with size 1232.7 KiB
25/06/05 10:01:21 WARN DAGScheduler: Broadcasting large task binary with size 1234.1 KiB
25/06/05 10:01:21 WARN DAGScheduler: Broadcasting large task binary with size 1235.5 KiB
25/06/05 10:01:21 WARN DAGScheduler: Broadcasting large task binary with size 1236.9 KiB
25/06/05 10:01:21 WAR

RMSE=0.2782463583445547


25/06/05 10:01:23 WARN DAGScheduler: Broadcasting large task binary with size 1315.6 KiB


In [36]:
df_reviews_score_based_reco = create_predictions_pandas_dataframe(model, df_sp_reviews_score, "profile", "anime_uid", "review_score")

display(df_reviews_score_based_reco.head())

25/06/05 10:01:56 WARN DAGScheduler: Broadcasting large task binary with size 1314.4 KiB
25/06/05 10:02:00 WARN DAGScheduler: Broadcasting large task binary with size 1305.8 KiB
25/06/05 10:02:11 WARN DAGScheduler: Broadcasting large task binary with size 1183.0 KiB
/var/folders/yt/qrbwlc0x6fj2rs4chrxkrm_m0000gn/T/ipykernel_47524/4173841574.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_recommendations[item_col] = df_recommendations[item_col].astype("int64")


,profile,anime_uid
0,-----noname-----,"[22789, 22135, 585, 4081, 4087, 20583, 28297, ..."
1,---SnowFlake---,"[34798, 2418, 22789, 523, 2034, 20583, 585, 16..."
2,---was-----,"[202, 7311, 5678, 1, 21327, 2924, 10162, 36296..."
3,--EYEPATCH--,"[34798, 12531, 523, 22789, 35839, 7674, 35180,..."
4,--Mizu--,"[17549, 4081, 4087, 523, 5678, 10162, 20583, 3..."


In [42]:
df_reviews_score_based_reco = pd.read_csv("../data/als_reviews_score_based_reco_before_clean.csv", index_col="profile")
df_reviews_score_based_reco = df_reviews_score_based_reco.rename(columns={"anime_uid": "recommendations"})
df_reviews_score_based_reco["recommendations"] = df_reviews_score_based_reco["recommendations"].apply(ast.literal_eval)

In [43]:
data_profiles = pd.read_csv("https://anime-recommendation-engine.s3.eu-west-3.amazonaws.com/data/profiles_clean.csv", index_col="profile")
data_profiles["favorites_anime"] = data_profiles["favorites_anime"].apply(ast.literal_eval)

In [44]:
def delete_favorites_from_recommendations(df_recommendations, df_favorites) :

    for profile in df_recommendations.index :
        favorites = df_favorites.at[profile,"favorites_anime"]
        recommendations = df_recommendations.at[profile,"recommendations"]

        recommendations = [value for value in recommendations if str(value) not in favorites]
        df_recommendations.at[profile, 'recommendations'] = recommendations

    return df_recommendations

In [45]:
df_reviews_score_based_reco = delete_favorites_from_recommendations(df_reviews_score_based_reco, data_profiles)

In [47]:
df_reviews_score_based_reco.to_csv("../data/als_reviews_score_based_reco.csv")